In [1]:
# IMPORT LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error as mse
from numpy import nan as na
import tensorflow as tf
import xgboost
import requests
import random
random.seed(6)
from numpy import isnan

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# LOAD FILES

In [3]:
ss = pd.read_csv('data/sample_submission.csv')
target_cols = ['Clearsky DHI','Clearsky DNI','Clearsky GHI'] 
ss

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
17515,0,0,0
17516,0,0,0
17517,0,0,0
17518,0,0,0


In [4]:
df = pd.read_csv('data/train.csv')
df['Timestamp'] = df.index.values
df.corr()['Clearsky DNI'].sort_values()

Solar Zenith Angle   -0.886323
Relative Humidity    -0.680471
Cloud Type           -0.073302
Month                -0.020091
Precipitable Water   -0.002180
Minute               -0.001348
Day                   0.000962
Wind Direction        0.006256
Fill Flag             0.006318
Pressure              0.007043
Timestamp             0.008620
Year                  0.010488
Dew Point             0.033601
Wind Speed            0.381162
Temperature           0.384353
Hour                  0.738537
Clearsky DHI          0.860988
Clearsky GHI          0.927926
Clearsky DNI          1.000000
Name: Clearsky DNI, dtype: float64

In [5]:
tdf = pd.read_csv('data/test.csv')
tdf['Timestamp'] = tdf.index.values + df['Timestamp'].max()
tdf = tdf[df.columns]
tdf

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Timestamp
0,2019,1,1,0,0,NaN,NaN,NaN,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,175295
1,2019,1,1,0,30,NaN,NaN,NaN,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,175296
2,2019,1,1,1,0,NaN,NaN,NaN,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,175297
3,2019,1,1,1,30,NaN,NaN,NaN,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0,175298
4,2019,1,1,2,0,NaN,NaN,NaN,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0,175299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,31,21,30,NaN,NaN,NaN,0,5.6,11.1,1006,68.64,77.89,0.8,276,2.2,0,192810
17516,2019,12,31,22,0,NaN,NaN,NaN,0,5.7,9.3,1006,78.31,83.06,0.8,274,1.7,0,192811
17517,2019,12,31,22,30,NaN,NaN,NaN,0,5.7,8.5,1006,82.64,88.34,0.8,273,1.8,0,192812
17518,2019,12,31,23,0,NaN,NaN,NaN,0,4.8,7.8,1006,81.49,94.37,0.8,272,1.8,0,192813


In [6]:
# APPEND TEST TO TRAIN SET

In [8]:
df = df.append(tdf)
df['Sunny'] = 0
df.Sunny[(df['Hour']>=10)&(df['Hour']<25)] = 1
df.Sunny[df['Hour']==0] = 1
df['Hour'] = df['Hour'] + (df['Minute']/60)
df.drop(['Minute'],axis=1, inplace=True)
df

,Year,Month,Day,Hour,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Timestamp,Sunny
0,2009,1,1,NaN,0.0,0.0,0.0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0,0,1
1,2009,1,1,NaN,0.0,0.0,0.0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0,1,0
2,2009,1,1,NaN,0.0,0.0,0.0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0,2,0
3,2009,1,1,NaN,0.0,0.0,0.0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0,3,0
4,2009,1,1,NaN,0.0,0.0,0.0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,31,21.5,NaN,NaN,NaN,0,5.6,11.1,1006,68.64,77.89,0.800,276.0,2.2,0,192810,1
17516,2019,12,31,22.0,NaN,NaN,NaN,0,5.7,9.3,1006,78.31,83.06,0.800,274.0,1.7,0,192811,1
17517,2019,12,31,22.5,NaN,NaN,NaN,0,5.7,8.5,1006,82.64,88.34,0.800,273.0,1.8,0,192812,1
17518,2019,12,31,23.0,NaN,NaN,NaN,0,4.8,7.8,1006,81.49,94.37,0.800,272.0,1.8,0,192813,1


In [9]:
# CONVERT ZENITH ANGLE TO RADS

In [10]:
df['Solar Zenith Angle'] = np.deg2rad(df['Solar Zenith Angle'])
df['Solar Zenith Angle * Relative Humidity'] = df['Relative Humidity'] * df['Solar Zenith Angle']
df['Day'] = (df.Timestamp - (df.Timestamp%48))/48

In [11]:
df

,Year,Month,Day,Hour,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Timestamp,Sunny,Solar Zenith Angle * Relative Humidity
0,2009,1,0.0,NaN,0.0,0.0,0.0,0,0.0,5.0,1010,75.34,1.852667,0.499,346.1,3.1,0,0,1,139.579932
1,2009,1,0.0,NaN,0.0,0.0,0.0,0,1.0,5.0,1010,80.81,1.959656,0.490,346.1,3.1,0,1,0,158.359776
2,2009,1,0.0,NaN,0.0,0.0,0.0,4,0.0,5.0,1010,78.27,2.068215,0.482,347.9,3.2,0,2,0,161.879201
3,2009,1,0.0,NaN,0.0,0.0,0.0,4,0.0,4.0,1010,78.27,2.177822,0.478,347.9,3.1,0,3,0,170.458115
4,2009,1,0.0,NaN,0.0,0.0,0.0,4,0.0,4.0,1010,76.45,2.288476,0.475,350.0,3.0,0,4,0,174.953968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,4016.0,21.5,NaN,NaN,NaN,0,5.6,11.1,1006,68.64,1.359437,0.800,276.0,2.2,0,192810,1,93.311753
17516,2019,12,4016.0,22.0,NaN,NaN,NaN,0,5.7,9.3,1006,78.31,1.449670,0.800,274.0,1.7,0,192811,1,113.523695
17517,2019,12,4016.0,22.5,NaN,NaN,NaN,0,5.7,8.5,1006,82.64,1.541824,0.800,273.0,1.8,0,192812,1,127.416324
17518,2019,12,4016.0,23.0,NaN,NaN,NaN,0,4.8,7.8,1006,81.49,1.647067,0.800,272.0,1.8,0,192813,1,134.219507


In [12]:
# HANDLE CATEGORICAL AND MISSING VALUES

In [13]:
df['Cloud'] = df['Cloud Type'].values 
df['Cloud Type'][df['Cloud Type']==10]=np.nan
df['Flag'] = df['Fill Flag'].values
df['Flag'][df['Flag']>10]=np.nan
df['Fill Flag'][df['Fill Flag']<0]=np.nan
df = pd.get_dummies(df,columns=['Cloud Type', 'Flag'])
df

,Year,Month,Day,Hour,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,Pressure,...,Cloud Type_4.0,Cloud Type_6.0,Cloud Type_7.0,Cloud Type_8.0,Cloud Type_9.0,Flag_0.0,Flag_1.0,Flag_3.0,Flag_4.0,Flag_7.0
0,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,5.0,1010,...,0,0,0,0,0,1,0,0,0,0
1,2009,1,0.0,NaN,0.0,0.0,0.0,1.0,5.0,1010,...,0,0,0,0,0,1,0,0,0,0
2,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,5.0,1010,...,1,0,0,0,0,1,0,0,0,0
3,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,4.0,1010,...,1,0,0,0,0,1,0,0,0,0
4,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,4.0,1010,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,4016.0,21.5,NaN,NaN,NaN,5.6,11.1,1006,...,0,0,0,0,0,1,0,0,0,0
17516,2019,12,4016.0,22.0,NaN,NaN,NaN,5.7,9.3,1006,...,0,0,0,0,0,1,0,0,0,0
17517,2019,12,4016.0,22.5,NaN,NaN,NaN,5.7,8.5,1006,...,0,0,0,0,0,1,0,0,0,0
17518,2019,12,4016.0,23.0,NaN,NaN,NaN,4.8,7.8,1006,...,0,0,0,0,0,1,0,0,0,0


In [14]:
# EXCLUDE SOME COLUMNS FROM TRAINING

In [15]:
drop_cols = ['Year','Month','Day','Timestamp']

In [16]:
# GENERATE HISTORICAL FEATURES & TRENDS
# FROM present hour / past hours  ONLY

In [17]:
cols = set(df.columns) - set(target_cols+drop_cols)
mid_cols = [('mid_'+x) for x in cols]
df[mid_cols] = df[cols].values 
for mid_col in mid_cols:
    df[mid_col][df['Hour']!=16]=np.nan
    df[mid_col].fillna(method='ffill',inplace=True)
df[mid_cols] = (df[cols].values) - (df[mid_cols].values)
df = df.replace({np.inf:np.nan,-1*np.inf:np.nan})
for col in cols:
    df['mean_'+col] = df[col].rolling(6).mean()
df['b'] = df['Relative Humidity'] - df['Relative Humidity'].shift(48)
df['e'] = df['Solar Zenith Angle'] - df['Solar Zenith Angle'].shift(48)
df['Temperature Trnd'] = df['Temperature'] - df['Temperature'].shift(1) 
df['Solar Zenith Angle Trnd'] = df['Solar Zenith Angle'] - df['Solar Zenith Angle'].shift(1) 
df['Relative Humidity Trnd'] = df['Relative Humidity'] - df['Relative Humidity'].shift(1) 
df['Precipitable Water Trnd'] = df['Precipitable Water'] - df['Precipitable Water'].shift(48) 
df['Dew Point Trnd'] = df['Dew Point'] - df['Dew Point'].shift(48) 
df['Wind Speed Trnd'] = df['Wind Speed'] - df['Wind Speed'].shift(1) 
df['Fill Flag Trnd'] = (df['Fill Flag']+1) / (df['Fill Flag'].shift(1)+1)
df['Pressure / Solar Zenith Angle'] = df['Pressure'] / df['Solar Zenith Angle']
df['Temperature / Solar Zenith Angle'] = df['Temperature'] / df['Solar Zenith Angle']
df['Dew Point * Temperature'] = df['Dew Point'] * df['Temperature']


In [18]:
df.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Clearsky DHI', 'Clearsky DNI',
       'Clearsky GHI', 'Dew Point', 'Temperature', 'Pressure',
       'Relative Humidity', 'Solar Zenith Angle', 'Precipitable Water',
       'Wind Direction', 'Wind Speed', 'Fill Flag', 'Timestamp', 'Sunny',
       'Solar Zenith Angle * Relative Humidity', 'Cloud', 'Cloud Type_0.0',
       'Cloud Type_1.0', 'Cloud Type_2.0', 'Cloud Type_3.0', 'Cloud Type_4.0',
       'Cloud Type_6.0', 'Cloud Type_7.0', 'Cloud Type_8.0', 'Cloud Type_9.0',
       'Flag_0.0', 'Flag_1.0', 'Flag_3.0', 'Flag_4.0', 'Flag_7.0',
       'mid_Precipitable Water', 'mid_Temperature', 'mid_Cloud Type_2.0',
       'mid_Cloud Type_9.0', 'mid_Flag_1.0', 'mid_Cloud Type_7.0',
       'mid_Cloud Type_0.0', 'mid_Solar Zenith Angle', 'mid_Cloud Type_1.0',
       'mid_Solar Zenith Angle * Relative Humidity', 'mid_Relative Humidity',
       'mid_Pressure', 'mid_Cloud', 'mid_Flag_0.0', 'mid_Cloud Type_4.0',
       'mid_Flag_7.0', 'mid_Cloud Type_8.0', 'mid

In [19]:
# Generate Cummulative Features from Training Date Only

In [20]:
df['Train'] = 1
df['Train'][df['Year']==df['Year'].max()] = np.nan
df

,Year,Month,Day,Hour,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,Pressure,...,Solar Zenith Angle Trnd,Relative Humidity Trnd,Precipitable Water Trnd,Dew Point Trnd,Wind Speed Trnd,Fill Flag Trnd,Pressure / Solar Zenith Angle,Temperature / Solar Zenith Angle,Dew Point * Temperature,Train
0,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,5.0,1010,...,NaN,NaN,NaN,NaN,NaN,NaN,545.160031,2.698812,0.00,1.0
1,2009,1,0.0,NaN,0.0,0.0,0.0,1.0,5.0,1010,...,0.106989,5.47,NaN,NaN,0.0,1.0,515.396663,2.551469,5.00,1.0
2,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,5.0,1010,...,0.108559,-2.54,NaN,NaN,0.1,1.0,488.343775,2.417543,0.00,1.0
3,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,4.0,1010,...,0.109607,0.00,NaN,NaN,-0.1,1.0,463.766127,1.836698,0.00,1.0
4,2009,1,0.0,NaN,0.0,0.0,0.0,0.0,4.0,1010,...,0.110654,-1.82,NaN,NaN,-0.1,1.0,441.341804,1.747888,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,4016.0,21.5,NaN,NaN,NaN,5.6,11.1,1006,...,0.085172,7.32,0.0,-3.2,-0.5,1.0,740.012250,8.165145,62.16,NaN
17516,2019,12,4016.0,22.0,NaN,NaN,NaN,5.7,9.3,1006,...,0.090234,9.67,0.0,-3.2,-0.5,1.0,693.950809,6.415251,53.01,NaN
17517,2019,12,4016.0,22.5,NaN,NaN,NaN,5.7,8.5,1006,...,0.092153,4.33,0.0,-3.2,0.1,1.0,652.474012,5.512951,48.45,NaN
17518,2019,12,4016.0,23.0,NaN,NaN,NaN,4.8,7.8,1006,...,0.105243,-1.15,0.0,-3.2,0.0,1.0,610.782602,4.735690,37.44,NaN


In [21]:
# Generate Statistical Predictions from Zenith Angle and Humidity Averages
# Using Train Set Only

In [22]:
df['Hour_Humidity'] = df['Relative Humidity'] * df['Train']
df['Hour_Humidity'] = df.groupby('Hour')['Hour_Humidity'].transform('mean')
df['PCT_Humidity'] = df['Relative Humidity'] - df['Hour_Humidity']

In [23]:
df = df[df['Year']>=df['Year'].min()+5]

In [24]:
for target in [target_cols[1]]:
    df['Hour_Target'+target] = df[target] * df['Train']
    df['Hour_Target'+target] = df.groupby('Hour')['Hour_Target'+target].transform('mean')

In [25]:
df = df.replace({np.inf:np.nan,-1*np.inf:np.nan})

In [26]:
 # TRAIN AND SUBMIT !!!!!!

In [27]:
tdf = df[df['Year']==df['Year'].max()]
df = df[df['Year']<df['Year'].max()]

In [28]:
drop_cols = ['Year','Month','Day','Train','Hour','Timestamp']

In [29]:
model = ExtraTreesRegressor(n_estimators=2000)
for target in target_cols:
    cols=df.drop(target_cols + drop_cols,axis=1).columns
    ss[target] = model.fit(df[cols].fillna(0), df[target]).predict(tdf[cols].fillna(0))
    #print(target, mse(ss[target],tdf[target]))
ss.to_csv('wipro.csv',index=False)
ss

ValueError: Length of values (35040) does not match length of index (17520)

In [28]:
print(df['Year'].nunique())

5


In [29]:
df

,Year,Month,Day,Hour,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,Pressure,...,Dew Point Trnd,Wind Speed Trnd,Fill Flag Trnd,Pressure / Solar Zenith Angle,Temperature / Solar Zenith Angle,Dew Point * Temperature,Train,Hour_Humidity,PCT_Humidity,Hour_TargetClearsky DNI
87648,2014,1,1826.0,0.0,0.0,0.0,0.0,3.0,5.0,1010,...,-6.0,0.3,1.000000,545.006002,2.698050,15.00,1.0,80.342519,10.167481,56.670318
87649,2014,1,1826.0,0.5,0.0,0.0,0.0,3.0,5.0,1010,...,-6.0,0.0,1.000000,515.258991,2.550787,15.00,1.0,83.372796,7.147204,5.729463
87650,2014,1,1826.0,1.0,0.0,0.0,0.0,3.0,5.0,1010,...,-6.0,0.0,1.000000,488.220175,2.416932,15.00,1.0,82.537100,5.662900,0.000000
87651,2014,1,1826.0,1.5,0.0,0.0,0.0,3.0,5.0,1010,...,-6.0,-0.1,1.000000,463.654654,2.295320,15.00,1.0,85.210219,2.979781,0.000000
87652,2014,1,1826.0,2.0,0.0,0.0,0.0,2.0,5.0,1010,...,-7.0,0.0,1.000000,441.240849,2.184361,10.00,1.0,84.796314,1.883686,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,2018,12,3651.0,21.5,51.0,555.0,168.0,19.4,20.8,1008,...,-1.5,-0.4,3.366667,741.769147,15.306347,403.52,1.0,65.147957,26.622043,641.497809
175292,2018,12,3651.0,22.0,37.0,388.0,84.0,19.1,20.1,1008,...,-0.8,-0.3,1.000000,695.581666,13.870230,383.91,1.0,68.536194,25.343806,545.393757
175293,2018,12,3651.0,22.5,15.0,115.0,18.0,19.1,19.6,1008,...,-0.4,-0.6,0.574257,653.919223,12.715096,374.36,1.0,73.038053,23.791947,398.908543
175294,2018,12,3651.0,23.0,0.0,0.0,0.0,18.7,19.2,1009,...,-0.3,-0.5,0.017241,612.798829,11.660790,359.04,1.0,75.561364,21.278636,261.131982


In [30]:
tdf

,Year,Month,Day,Hour,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Temperature,Pressure,...,Dew Point Trnd,Wind Speed Trnd,Fill Flag Trnd,Pressure / Solar Zenith Angle,Temperature / Solar Zenith Angle,Dew Point * Temperature,Train,Hour_Humidity,PCT_Humidity,Hour_TargetClearsky DNI
0,2019,1,3651.0,0.0,NaN,NaN,NaN,18.4,18.8,1008,...,0.0,0.2,1.0,543.670769,10.139891,345.92,NaN,80.342519,17.357481,56.670318
1,2019,1,3652.0,0.5,NaN,NaN,NaN,18.4,18.6,1008,...,0.1,0.2,1.0,514.009841,9.484705,342.24,NaN,83.372796,15.547204,5.729463
2,2019,1,3652.0,1.0,NaN,NaN,NaN,18.2,18.5,1008,...,0.1,0.3,1.0,487.047949,8.938876,336.70,NaN,82.537100,15.812900,0.000000
3,2019,1,3652.0,1.5,NaN,NaN,NaN,18.2,18.3,1008,...,0.2,0.2,1.0,462.551223,8.397507,333.06,NaN,85.210219,14.369781,0.000000
4,2019,1,3652.0,2.0,NaN,NaN,NaN,18.0,18.0,1008,...,0.1,0.1,1.0,440.199282,7.860701,324.00,NaN,84.796314,14.913686,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,4016.0,21.5,NaN,NaN,NaN,5.6,11.1,1006,...,-3.2,-0.5,1.0,740.012250,8.165145,62.16,NaN,65.147957,3.492043,641.497809
17516,2019,12,4016.0,22.0,NaN,NaN,NaN,5.7,9.3,1006,...,-3.2,-0.5,1.0,693.950809,6.415251,53.01,NaN,68.536194,9.773806,545.393757
17517,2019,12,4016.0,22.5,NaN,NaN,NaN,5.7,8.5,1006,...,-3.2,0.1,1.0,652.474012,5.512951,48.45,NaN,73.038053,9.601947,398.908543
17518,2019,12,4016.0,23.0,NaN,NaN,NaN,4.8,7.8,1006,...,-3.2,0.0,1.0,610.782602,4.735690,37.44,NaN,75.561364,5.928636,261.131982


In [31]:
ss

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,0.0000,0.0000,0.0000
1,0.0000,0.0000,0.0000
2,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000
4,0.0000,0.0000,0.0000
...,...,...,...
17515,51.3010,622.2890,177.8655
17516,35.6905,468.6935,88.1955
17517,13.5715,170.1820,18.1360
17518,0.0035,0.0260,0.0070


In [32]:
df.corr()[target_cols[1]].sort_values().head(20)

mid_Solar Zenith Angle                        -0.901176
Hour_Humidity                                 -0.900948
mid_Solar Zenith Angle * Relative Humidity    -0.900772
Solar Zenith Angle                            -0.888941
Solar Zenith Angle * Relative Humidity        -0.884592
mean_Solar Zenith Angle                       -0.844309
mean_Solar Zenith Angle * Relative Humidity   -0.806522
mid_Relative Humidity                         -0.711599
Relative Humidity                             -0.689510
mean_Relative Humidity                        -0.553305
Relative Humidity Trnd                        -0.242182
mean_Cloud Type_1.0                           -0.156890
Cloud Type_1.0                                -0.142056
mean_Cloud Type_7.0                           -0.129968
mean_Cloud                                    -0.127653
mid_Cloud Type_1.0                            -0.118930
Wind Speed Trnd                               -0.111879
Cloud Type_7.0                                -0

In [33]:
df.corr()[target_cols[1]].sort_values().tail(20)

mean_Temperature                    0.296547
Temperature Trnd                    0.355650
mid_Wind Speed                      0.376505
Wind Speed                          0.377861
mean_Wind Speed                     0.380767
Temperature                         0.388974
Temperature / Solar Zenith Angle    0.575188
Pressure / Solar Zenith Angle       0.652706
mean_Hour                           0.669958
Sunny                               0.693795
mid_Sunny                           0.693795
mid_Temperature                     0.719866
Hour                                0.740401
mid_Hour                            0.740401
mean_Sunny                          0.752017
Clearsky DHI                        0.872980
Clearsky GHI                        0.928606
Hour_TargetClearsky DNI             0.970988
Clearsky DNI                        1.000000
Train                                    NaN
Name: Clearsky DNI, dtype: float64